<a href="https://colab.research.google.com/github/kevinscaria/InstructABSA/blob/main/JointTask_Training_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [ ]:
#import pandas as pd
#import json

# Load the CSV file
#df = pd.read_csv('/home/ec2-user/SageMaker/InstructABSA-main/Dataset/senti.csv')

# Process the 'Decisions' column
#df['Decisions'] = df['Decisions'].apply(json.loads)  # convert string to dictionary
#df['aspectTerms'] = df['Decisions'].apply(lambda d: [{'term': k, 'polarity': v} for k, v in d.items()])  # convert dictionary to desired format
#df = df.drop(columns=['Decisions'])  # remove the original 'Decisions' column

# Rename the 'Title' column to 'raw_text'
#df = df.rename(columns={'Title': 'raw_text'})

# Save the DataFrame to a new CSV file
#df.to_csv('/home/ec2-user/SageMaker/InstructABSA-main/Dataset/sentiv1.csv', index=False)


In [3]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sentencepiece
#!pip install --upgrade torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 20.3 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 14.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.1 MB/s eta 0:00:00:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 26.3 M

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

In [2]:
import torch
print(torch.__version__)

1.13.1+cu117


## Training

In [3]:
import os
task_name = 'joint_task'
experiment_name = 'lapt2014_iabsa1'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  lapt2014_iabsa1
Model output path:  ./Models/joint_task/allenaitk-instruct-base-def-pos-lapt2014_iabsa1


In [4]:
from sklearn.model_selection import train_test_split
import ast
# Load the data
id_data_file_path = '/home/ec2-user/SageMaker/InstructABSA-main/Dataset/sentiv1.csv'
id_df = pd.read_csv(id_data_file_path)


In [5]:
# Convert string representation of dictionaries into actual dictionaries
#id_df['Decisions'] = id_df['Decisions'].apply(ast.literal_eval)
import ast
# Split the data into train and test datasets
id_train_df, id_test_df = train_test_split(id_df, test_size=0.2, random_state=42)  # 80% for training, 20% for testing



#id_train_df['Decisions'] = id_train_df['Decisions'].apply(try_eval)
#id_test_df['Decisions'] = id_test_df['Decisions'].apply(try_eval)

In [6]:

# Check and see how many problematic rows there are
#print(id_train_df['Decisions'].isna().sum())
#print(id_test_df['Decisions'].isna().sum())
# Print some problematic rows
#print(id_train_df[id_train_df['Decisions'].isna()]['Decisions'].head(10))

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_train_df, id_test_df)

if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [7]:
# Load the data
#id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
#id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
#id_tr_df = pd.read_csv(id_train_file_path)
#id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
#instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
#instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
#loader = DatasetLoader(id_tr_df, id_te_df)
#if loader.train_df_id is not None:
#    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
#if loader.test_df_id is not None:
#    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [8]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)
use_mps = False  # or False, depending on your requirement

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':32,
    'num_train_epochs':1,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495M [00:00<?, ?B/s]

Map:   0%|          | 0/8602 [00:00<?, ? examples/s]

Map:   0%|          | 0/2151 [00:00<?, ? examples/s]

In [9]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: aspectTerms, S No., Words, text, __index_level_0__, raw_text. If aspectTerms, S No., Words, text, __index_level_0__, raw_text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 8602
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 538
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## Inference

In [ ]:
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [11]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)
if not torch.cuda.is_available():
    t5_exp.device = 'cpu'
# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file ./Models/joint_task/allenaitk-instruct-base-def-pos-lapt2014_iabsa1/config.json
Model config T5Config {
  "_name_or_path": "./Models/joint_task/allenaitk-instruct-base-def-pos-lapt2014_iabsa1",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dty

Map:   0%|          | 0/8602 [00:00<?, ? examples/s]

Map:   0%|          | 0/2151 [00:00<?, ? examples/s]

Model loaded to:  cpu


100%|██████████| 538/538 [41:29<00:00,  4.63s/it]


Model loaded to:  cpu


100%|██████████| 135/135 [10:29<00:00,  4.66s/it]


In [12]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.7597027250206441
Train Recall:  0.7189372232352175
Train F1:  0.7387580299785866
Test Precision:  0.7597845601436266
Test Recall:  0.7291523087525844
Test F1:  0.7441533321610692


In [ ]:
model_checkpoint = '/full/path/to/InstructABSA-main/Models/joint_task'
self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [15]:
!python run_model.py -mode cli -task joint \
-model_checkpoint InstructABSA-main/Models/joint_task/allenaitk-instruct-base-def-pos-lapt2014_iabsa1 \
-test_input 'The cab ride was amazing but the service was pricey'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Task:  joint
Mode set to:  Individual sample inference
Running inference on input:  The cab ride was amazing but the service was pricey
1
Traceback (most recent call last):
  File "run_model.py", line 109, in <module>
    t5_exp = T5Generator(model_checkpoint)
  File "/home/ec2-user/SageMaker/InstructABSA-main/InstructABSA/utils.py", line 15, in __init__
    self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  File "/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/transformers/models/auto/tokenization_auto.py", line 582, in from_pretrained
    tokenizer_config = get_tokenizer_config(pretrained_model_name_or_path, **kw